In [14]:
# !pip install transformers[torch]
# !pip install einops
# !pip install datasets
import numpy as np
import pandas as pd


In [1]:
import torch
import transformers # import LMHeadModel, Tokenizer, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments
from transformers import AutoTokenizer

In [15]:
from datasets import load_dataset

In [5]:
model_name = 'mosaicml/mpt-1b-redpajama-200b-dolly'
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

C:\Users\rraj0\.conda\envs\clone_cs285_2\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\rraj0\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [6]:
model = transformers.AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)

A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-1b-redpajama-200b-dolly:
- configuration_mosaic_gpt.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-1b-redpajama-200b-dolly:
- low_precision_layernorm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-1b-redpajama-200b-dolly:
- attention.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-1b-redpajama-200b-dolly:
- gpt_blocks.py
- low_precision_layernorm.py
- attention.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-1b-redpajama-200b-dolly:
- param_init_fns.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-1b-redpajama-200b-dolly:
- mosaic_gpt.py
- gpt_blocks.py
- param_init_fns.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


You are using config.init_device='cpu', but you can also use config.init_device="meta" with Composer + FSDP for fast initialization.


In [8]:
model.to("cuda")

In [9]:
tokenizer.pad_token = tokenizer.eos_token  # Set the pad token

In [10]:
import torch

def tokenize_function(example):
    # Tokenize the instruction text
    model_inputs = tokenizer(example['instruction'], truncation=True, padding='max_length')

    # Convert attention_mask and input_ids to tensors
    model_inputs = {key: torch.tensor(val) for key, val in model_inputs.items()}

    # Now you can convert the attention_mask to boolean
    if 'attention_mask' in model_inputs:
        model_inputs['attention_mask'] = model_inputs['attention_mask'].bool()

    return model_inputs


In [18]:
dataset = load_dataset("NumbersStation/NSText2SQL")


In [20]:
# Define the number of rows you want to print
num_rows_to_print = 5

# Print the first few rows of the 'source' column
for i in range(num_rows_to_print):
    print(dataset['train'][i]['source'])

In [23]:
# Assuming 'spider' is identified by a field in the dataset
# Replace 'field_name' and 'spider' with the actual field name and value that identifies the spider subset
dataset = dataset.filter(lambda example: example['source'] != 'spider')

In [24]:
# Define the number of rows you want to print
num_rows_to_print = 5

# Print the first few rows of the 'source' column
for i in range(num_rows_to_print):
    print(dataset['train'][i]['source'])

In [26]:


# Print all available splits in the dataset
print("Available splits in the dataset:", list(dataset.keys()))


Available splits in the dataset: ['instruction', 'output', 'source']


In [27]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

AttributeError: 'DataFrame' object has no attribute 'map'

In [ ]:
# Apply the tokenization function to each example in your dataset
# need this if still testing tokenizer
# tokenized_datasets = dataset.map(tokenize_function, batched=True, load_from_cache_file=False)


Map:   0%|          | 0/282294 [00:00<?, ? examples/s]

In [ ]:
# import torch
# torch.cuda.empty_cache()

In [ ]:
# Create a data collator
data_collator = transformers.DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False,
)

In [ ]:
# Define training arguments
training_args = transformers.TrainingArguments(
    output_dir="./english_to_sql_finetuned",
    overwrite_output_dir=True,
    num_train_epochs=3,  #the epochs can be adjusted
    per_device_train_batch_size=8,
    save_steps=10,
    save_total_limit=5,
    evaluation_strategy="steps",
    eval_steps=10_000,
)

In [ ]:
from transformers import Trainer, TrainingArguments
from transformers.modeling_outputs import CausalLMOutputWithPast

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        # Remove the 'labels' from inputs if your model doesn't accept them
        labels = inputs.pop("labels", None)

        # Forward pass
        outputs = model(**inputs)

        # Compute the loss by comparing the outputs (logits) with the shifted labels
        logits = outputs.logits
        # Shift the labels to the right to align with model's output for causal language modeling
        shift_logits = logits[..., :-1, :].contiguous()
        shift_labels = labels[..., 1:].contiguous()
        # Flatten the logits and labels to compute loss
        loss_fct = torch.nn.CrossEntropyLoss()
        loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))

        return (loss, outputs) if return_outputs else loss




In [ ]:
# Initialize your custom trainer with the custom loss computation
trainer = CustomTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets["train"],
    # If you have an eval dataset, include it here
    # eval_dataset=tokenized_datasets["validation"] if "validation" in tokenized_datasets else None,
)

In [ ]:
# Now, when you call train, it will use your custom loss computation
trainer.train()

# Save the model and tokenizer
model.save_pretrained('./model_output')
tokenizer.save_pretrained('./model_output')

NameError: ignored

In [ ]:
# Save the fine-tuned model
model.save_pretrained("./english_to_sql_finetuned")

In [ ]:
tokenizer.vocab_size

50254